In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error


import warnings
warnings.filterwarnings('ignore')

## Step 1: Loading data from file into Dataframe

In [24]:
housing_data = pd.read_csv('train.csv')
housing_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Step 2: Perform EDA

In [25]:
housing_data.shape

(1460, 81)

1460 Row X 81 Columns

In [26]:
# Checking column types of dataset.
housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [27]:
# Check number of columns for each data type.
housing_data.dtypes.value_counts()

object     43
int64      35
float64     3
Name: count, dtype: int64

#### Observation
- The largest number of columns are of object type.
- The dataset does not have any column of type `Boolean`.

According to data dictionary, there are a few columns in dataset which have numeric values but they represent the categorical information.


Let's create list of names of such columns.

In [28]:
numerica_cat_cols = ['YearBuilt','YearRemodAdd', 'GarageYrBlt','YrSold','OverallQual', 
                     'OverallCond','MSSubClass','Fireplaces','GarageCars','MoSold',
                     'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 
                     'KitchenAbvGr', 'TotRmsAbvGrd']

### Let's take a glance at missing values in the dataset.

In [29]:
# Number of columns with missing values.
cols_with_missing_values = pd.DataFrame(housing_data.isnull().sum())[housing_data.isnull().sum()>0].sort_values(by=[0])

# Renaming column '0' to count
cols_with_missing_values.rename(columns={0: 'count'}, inplace=True)

# Adding new column percentage with ratio of missing value for each column.
cols_with_missing_values['percentage'] = cols_with_missing_values['count'].apply(lambda x: x / len(housing_data) * 100)

print(cols_with_missing_values)
print('Number of columns with missing values: {}'.format(len(cols_with_missing_values)))
print('Number of columns with number of missing values ratio > 5%: {}'.format(len(cols_with_missing_values[cols_with_missing_values['percentage'] > 5])))

              count  percentage
Electrical        1    0.068493
MasVnrArea        8    0.547945
BsmtQual         37    2.534247
BsmtCond         37    2.534247
BsmtFinType1     37    2.534247
BsmtExposure     38    2.602740
BsmtFinType2     38    2.602740
GarageCond       81    5.547945
GarageQual       81    5.547945
GarageFinish     81    5.547945
GarageYrBlt      81    5.547945
GarageType       81    5.547945
LotFrontage     259   17.739726
FireplaceQu     690   47.260274
MasVnrType      872   59.726027
Fence          1179   80.753425
Alley          1369   93.767123
MiscFeature    1406   96.301370
PoolQC         1453   99.520548
Number of columns with missing values: 19
Number of columns with number of missing values ratio > 5%: 12


#### Observations
- 19 out of 81 columns have missing values in them.
- 7 columns with missing value count > 100.
- 12 columns have more than 5% values missing in them.

### Anlyzing Numeric data from the dataset.

In [30]:
# Let's analyze columns with numeric data.

numeric_data = housing_data.select_dtypes(include=['int64', 'float64'])
numeric_data.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [31]:
numeric_data.shape

(1460, 38)

#### Let's deal with missing values in numeric columns.

In [32]:
# Let's checks missing values numeric columns.

pd.DataFrame(numeric_data.isnull().sum())[numeric_data.isnull().any()]

,0
LotFrontage,259
MasVnrArea,8
GarageYrBlt,81


#### Observation
- Columns LotFrontage, MasVnrArea, GarageYrBlt have missing values.
- Missing values of `LotFrontage` and `MasVnrArea` can be imputed with value 0.

In [33]:
housing_data['LotFrontage'].fillna(0, inplace=True)
housing_data['LotFrontage'].isnull().sum()

0

In [34]:
housing_data['MasVnrArea'].fillna(0, inplace=True)
housing_data['MasVnrArea'].isnull().sum()

0

Columns `YearBuilt`, `YearRemodAdd`, `GarageYrBlt` and `YrSold` have exact year as values. 

These columns can be converted to number of years by deducting them from the current year 2024.

In [35]:
year_cols = ['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']

for col in year_cols:
    housing_data[col] = 2024 - housing_data[col]